## Re-forcasting example

This is re-forecasting workflow that runs for 72h forecasting every 12 hours using D3D.

In [ ]:
#to use the full width of the browser window
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pyPoseidon.utils import cast,data,point
import pandas as pd
import datetime
import pickle
import xarray as xr

### setup

In [ ]:
#read the info from the first run
with open('/Users/brey/Downloads/EUR/D3D/20100201.00/eur025_info.pkl', 'r') as f:
              info=pickle.load(f)

In [ ]:
#info

In [ ]:
#define some info
info.update({'path':'/Users/brey/Downloads/EUR/D3D/', # The path of the project
     'case':'European 2010', # a reference tag
     })

In [ ]:
#creating a time sequence of the runs
start_date = pd.to_datetime('2010-2-1 0:0:0')
end_date = pd.to_datetime('2010-2-2 12:0:0')
date_list = pd.date_range(start_date,end_date, freq='12H')
date_list

In [ ]:
#append to dic
info.update({'start_date':start_date,'end_date':end_date, 'dates' : date_list})

In [ ]:
#creating a sequence of folder to store the runs. In this case we name them after the date attribute.
#NOTE that the first folder is the fisrt run already perfomed!!
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in date_list]
info.update({'folders':folders})
folders

In [ ]:
#creating a sequence of folder from which we read the meteo.
meteo = []
PATH='/Users/brey/Downloads/'
for date in date_list:
    end_date= pd.to_datetime(date) + pd.to_timedelta(info['time_frame'])
    end_date = end_date.strftime(format='%Y-%m-%d %H:%M:%S')
    dr = pd.date_range(date, end_date, freq='73H')
    dur = [PATH+'{:04d}/{:02d}/{:02d}/'.format(x.year,x.month,x.day)+datetime.datetime.strftime(x, '%Y%m%d.%H')+'.tropical_cyclone.grib' for x in dr]
    meteo.append(dur)

In [ ]:
meteo

In [ ]:
info.update({'meteo_files':meteo})

In [ ]:
#check
info['folders'],info['meteo_files']

In [ ]:
#check dict
#info

### run the hindcast

In [ ]:
#set cast
h = cast.cast(**info) # initialize

In [ ]:
h.run()

## Analysis of data

In [ ]:
%matplotlib inline

In [ ]:
folders = [info['path']+f for f in info['folders']]

In [ ]:
folders

In [ ]:
out = []
for folder in folders:
    out.append(data(folders=[folder],solver='d3d'))


In [ ]:
out[0].impl.dem.bathymetry.plot()

In [ ]:
out[0].impl.obs.locations

In [ ]:
 plat,plon = out[0].impl.obs.locations.loc[6,['lat','lon']]

In [ ]:
p6=[]
for i in range(len(out)):
    ts = point(lon=plon,lat=plat,data=out[i].impl)
    ts.tseries(var='S1')
    p6.append(ts.S1)

In [ ]:
p = out[0].impl.obs.iloc(6)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,10))
ax=p['Storm Surge'].plot()
for i in range(len(out)):
    p6[i].plot(ax=ax)
plt.legend(['tide gauge'] + info['folders'])


### Create a hindcast dataset from re-forecasting data

In [ ]:
out[0].impl.Dataset[dict(time=slice(None, 12))]

In [ ]:
xars=[]
for i in range(len(out)):
    xars.append(out[i].impl.Dataset[dict(time=slice(None, 12))])

In [ ]:
g=xr.merge(xars)

In [ ]:
g

In [ ]:
hin=out[0].impl # copy the data structure

In [ ]:
hin.Dataset=g #assign the new hindcast dataset

In [ ]:
hp = point(lon=plon,lat=plat,data=hin)
hp.tseries(var='S1')

In [ ]:
hp.S1.plot()

In [ ]:
ph = hin.obs.iloc(6)

In [ ]:
## Join the graphs
ax = ph.plot(figsize=(14,10))
hp.S1.plot(ax=ax,color=['r'])
ax.legend(list(ph.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))